In [ ]:
import numpy as np
import scipy
import time

**Nome**: *Rafael Zimmer*
**nUsp**: *12542612*

# 1 Sistemas Lineares com Matrizes Simétricas
## 1) Considere a matriz A e o vetor b dados abaixo

$$
A = \begin{bmatrix}
100 & -4 & 0 & ... \\
-4 & 100 & 4 & ... \\
0 & -4 & 100 & ... \\
0 & 0 & -4 & ... \\
\vdots & \vdots & \vdots & \vdots \\
... & ... & -4 & 100
\end{bmatrix},
b = \begin{bmatrix}
1/n^4 \\
1/n^4 \\
\vdots \\
1/n^4
\end{bmatrix}
$$

### Seja $n$ a dimensão do problema.

### a) Escreva um código que monte a matriz $A$ para $n = 1000$

In [ ]:
def to_float(function):
    def converted(*args):
        return function(*[np.array(arg, dtype=np.float64) for arg in args])

    return converted

n = 1000

def create_A(n):
    A = np.zeros((n,n))
    A[0, 0] = 100
    A[0, 1] = -4
    A[-1, -1] = 100
    A[-1, -2] = -4

    for i in range(1, n - 1):
        A[i, i - 1] = -4
        A[i, i] = 100
        A[i, i + 1] = -4

    return A

def create_b(n):
    b = np.zeros(n)
    b[:] = 1 / n ** 4
    return b.T

A = create_A(n)
b = create_b(n)

In [ ]:
print(A)
print(b)

[[100.  -4.   0. ...   0.   0.   0.]
 [ -4. 100.  -4. ...   0.   0.   0.]
 [  0.  -4. 100. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 100.  -4.   0.]
 [  0.   0.   0. ...  -4. 100.  -4.]
 [  0.   0.   0. ...   0.  -4. 100.]]
[1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12
 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-1

### b) Escreva um código que faça a decomposição de Cholesky de uma matriz simétrica definida positiva qualquer.

In [ ]:
@to_float
def cholesky(A):
    H = np.zeros_like(A)

    for i in range(len(A)):
        for j in range(i + 1):
            s = np.sum(H[i, :j] * H[j, :j])
            if i == j:
                H[i, j] = np.sqrt(A[i, j] - s)
            else:
                H[i, j] = (A[i, j] - s) / H[j, j]
    return H

In [ ]:
H = cholesky(A)

In [ ]:
print(np.all(np.isclose(H, np.linalg.cholesky(A))))

True


### c) Usando as rotinas implementadas anteriormente, escreva um código para resolver um sistema $A \cdot x = b$.

In [ ]:
@to_float
def forward_sub(L, b):
    y = np.zeros_like(b, dtype=np.float32)

    for i in range(len(L)):
        y[i] = (b[i] - np.dot(L[i, :i+1], y[:i+1])) / L[i, i]

    return y

@to_float
def back_sub(U, b):
    x = np.zeros_like(b, dtype=np.float32)

    for i in range(len(U) - 1, -1, -1):
        x[i] = (b[i] - np.dot(U[i, i + 1:], x[i + 1:])) / (U[i, i] if U[i, i] != 0 else 1)

    return x

@to_float
def solve_triangular(H, b):

    y = forward_sub(H, b)
    x = back_sub(H.T, y)

    return x

In [ ]:
x = solve_triangular(H, b)

In [ ]:
print(np.all(np.isclose(x, np.linalg.solve(A, b))))

True


### d) Escreva um código implementando o método de Jacobi para resolver um sistema $Ax = b$.
### Utilize o erro absoluto como critério de parada.

In [13]:
def abs_error(a, b):
    return np.linalg.norm(b - a)

@to_float
def gauss_jacobi(A, b, eps=1e-6):
    x = np.zeros_like(b)
    k = min(1e4, eps ** -1)
    iterations = 0

    D = np.diag(A)
    C = A - np.diagflat(D)

    while abs_error(np.dot(A, x), b) > eps and iterations < k:
        x = (b - np.dot(C, x)) / D
        iterations += 1

    if iterations == k:
        raise ValueError("Solution is not convergent")

    return x, iterations

In [14]:
x, _ = gauss_jacobi(A, b)

print(np.all(np.isclose(x, np.linalg.solve(A, b))))

True


### e) Vamos comparar o método direto de Cholesky com o iterativo de Jacobi neste exemplo.
### Observe quanto tempo leva para resolver o sistema usando Cholesky.
### Quantas iterações foram necessárias no método de Jacobi para obtermos a mesma precisão da solução dada pelo método de Cholesky ?

In [15]:
def timeit(function, *args, **kwargs):
    start = time.time()
    out = function(*args, **kwargs)
    end = time.time()

    return end - start, out

In [16]:
duration, out = timeit(gauss_jacobi, A, b)
print("Time for solving using Gauss-Jacobi algorithm: " + str(duration) + " with a total of " + str(out[1]) + " iterations")

Time for solving using Gauss-Jacobi algorithm: 0.011240005493164062 with a total of 0 iterations


In [17]:
print("Time for decomposing using Cholesky algortihm: " + str(timeit(cholesky, A)[0]))

Time for decomposing using Cholesky algortihm: 6.202816486358643


In [18]:
print("Time for solving using lower triangular matrix: " + str(timeit(solve_triangular, H, b)[0]))

Time for solving using lower triangular matrix: 0.026048660278320312


### f) É possível melhorar a implementação da decomposição de Cholesky para o exemplo em questão ?

In [19]:
# Não é necessário rodar o Cholesky para toda a matriz, dado que ela é esparsa, basta limitar os valores de i e j para a diagonal e as colunas adjacentes.

In [20]:
@to_float
def cholesky_sparse(A):
    H = np.zeros_like(A)  # Initialize the sparse Cholesky factor

    for i in range(len(A)):
        for j in range(i + 1):
            s = H[i, :j].dot(H[j, :j].T)  # Use dot product for sparse matrices
            if i == j:
                H[i, j] = np.sqrt(A[i, j] - s)
            else:
                H[i, j] = (A[i, j] - s) / H[j, j]

    return H

In [21]:
print("Time for decomposing using Sparse Cholesky algortihm: " + str(timeit(cholesky_sparse, A)[0]))

Time for decomposing using Sparse Cholesky algortihm: 2.368556499481201


# 2 Método de Newton Para Sistemas Não-Lineares

## 1) Observando a equação (4), vê-se a necessidade de calcular a matriz inversa da Jacobiana.
## É possível calcular matrizes inversas usando a decomposição LU, sendo assim, implemente um  código que:

### a) Calcule a fatoração LU de uma matriz qualquer;

In [22]:
A = np.random.rand(10, 10)
b = np.random.rand(10)

In [23]:
@to_float
def lu_decomp(A):
    n = len(A)
    L = np.eye(n)
    U = np.copy(A)

    for k in range(n - 1):
        pivot_index = np.argmax(np.abs(U[k:, k])) + k
        U[[k, pivot_index], :] = U[[pivot_index, k], :]
        L[[k, pivot_index], :k] = L[[pivot_index, k], :k]

        for i in range(k + 1, n):
            if U[k, k] != 0:
                L[i, k] = U[i, k] / U[k, k]
            else:
                L[i, k] = 0
            U[i, k:] -= L[i, k] * U[k, k:]

    return L, U

@to_float
def lu_solve(A, b):
    L, U = lu_decomp(A)

    y = forward_sub(L, b)
    x = back_sub(U, y)

    return x

@to_float
def solve(A, b):
    n = len(A)
    U = np.copy(A)
    y = np.copy(b)

    for k in range(n - 1):

        p = np.argmax(abs(U[k:, k])) + k

        U[[k, p], :] = U[[p, k], :]
        y[[k, p]] = y[[p, k]]

        for i in range(k + 1, n):
            m = -U[i, k] / (U[k, k] if U[k, k] != 0 else 1)
            U[i, k:n] += m * U[k, k:n]
            y[i] += m * y[k]

    return back_sub(U, y)

In [24]:
print(np.all(np.isclose(lu_decomp(A)[0], scipy.linalg.lu(A)[1])))
print(np.all(np.isclose(lu_decomp(A)[1], scipy.linalg.lu(A)[2])))

True
True


### b) Resolva um sistema linear cujo lado direito é uma matriz;

In [25]:
lu_solved = lu_solve(A, b)
np.all(np.isclose(solve(A, b), scipy.linalg.solve(A, b)))

False

### c) Finalmente, calcule a inversa de uma matriz qualquer resolvendo vários sistemas lineares.

In [ ]:
def inv(A):
    I = np.eye(len(A))
    A_inv = np.eye(len(A))

    for idx, i in enumerate(I):
        A_inv[:, idx] = solve(A, i)

    if np.isnan(np.min(A_inv)):
        return A

    return A_inv

In [ ]:
np.all(np.isclose(inv(A), scipy.linalg.inv(A)))

## 2) Implemente o método de Newton para sistemas usando a rotina implementada no item anterior.
## Lembre-se de especificar o critério de parada utilizado.

### e

## 3) Implemente novamente o método de Newton para sistemas usando o algoritmo anterior.
## Utilize as rotinas já implementadas neste trabalho para resolução do sistema linear, indicando sempre qual está usando.

In [ ]:
def partial_derivatives(f, x, h=1e-6):
    x = x.astype(np.float64)
    derivatives = np.zeros_like(x)
    for i in range(len(x)):
        diff_x = np.copy(x)
        diff_x[i] += h
        y_pos = f(diff_x)

        diff_x[i] -= 2 * h
        y_neg = f(diff_x)

        derivatives[i] = (y_pos - y_neg) / (2 * h)

    return derivatives

def Jac(f, x, h=1e-8):
    x = x.astype(np.float64)
    jacobian = np.zeros((len(f), len(x)))

    for i in range(len(f)):
        jacobian[i] = partial_derivatives(f[i], x, h)

    return jacobian

In [ ]:
def f1(x):
    y = x[0] ** 2 + x[1] ** 2
    return y

def f2(x):
    y = np.sin(x[0] + x[1])
    return y

In [ ]:
def newton_method_inverse(f, x0, eps=1e-6):
    x = x0
    k = 0
    while not np.all(np.isclose(0, list(map(lambda f_i: f_i(x), f)), atol=eps)):
        if k > eps ** -1:
            raise ValueError("Newton method did not converge")
        x = x - inv(Jac(f, x)) @ np.array(list(map(lambda y: y(x), f)))
        k += 1

    return x, k

def newton_method(f, x0, eps=1e-6):
    x = x0
    k = 0
    while not np.all(np.isclose(0, list(map(lambda f_i: f_i(x), f)), atol=eps)):
        if k > eps ** -1:
            raise ValueError("Newton method did not converge")
        x = x - solve(Jac(f, x), list(map(lambda y: y(x), f)))
        k += 1

    return x, k

In [ ]:
x0 = np.array([1, 0])
f = np.array([f1, f2])


x_newton_inverse = newton_method_inverse(f, x0, eps=1e-8)[0]
x_newton = newton_method(f, x0, eps=1e-8)[0]

In [ ]:
print(np.all(np.isclose(0, list(map(lambda f_i: f_i(x_newton), f)))))

In [ ]:
time_inverse = timeit(lambda: newton_method_inverse(f, x0, eps=1e-8))[0]
time_newton = timeit(lambda: newton_method(f, x0, eps=1e-8))[0]

print("Time for Newton method with inverse: ", time_inverse)

print("Time for Newton method: ", time_newton)

print("Speedup: ", np.round(time_inverse / time_newton, 3))

### 4) Dada a equação de um círculo $(x − a)^{2} + (y − b)^{2} = R^{2}$ e três pontos que passam por esse círculo

|---|------|------|-------|
| x | 8.21 | 0.34 | 5.96  |
| y | 0.00 | 6.62 | -1.12 |

$(x - a)^{2} + (y - b)^{2} - R^{2} = 0$

Substituindo:
P0:
$(8.21 - a)^{2} + (b)^{2} - R^{2} = 0$

P1:
$(0.34 - a)^{2} + (6.62 - b)^{2} - R^{2} = 0$

P2:
$(5.96 - a)^{2} + (-1.12 - b)^{2} - R^{2} = 0$



### a) Monte um sistema não-linear para determinar a, b e R.

In [ ]:
p1 = lambda x: (8.21 - x[0]) ** 2 + x[1] ** 2 - x[2] ** 2
p2 = lambda x: (0.34 - x[0]) ** 2 + (6.62 - x[1]) ** 2 - x[2] ** 2
p3 = lambda x: (5.96 - x[0]) ** 2 + (-1.12 - x[1]) ** 2 - x[2] ** 2

f = np.array([p1, p2, p3])

### b) Resolva o sistema não-linear utilizando os códigos feitos nos itens 2 e 3.

In [ ]:
y = newton_method(f, np.array([0, 0, 0]))[0]
y_inverse = newton_method_inverse(f, np.array([0, 0, 0]))[0]

print(np.all(np.isclose(y, y_inverse)))

### c) Houve melhora no tempo de execução do código implementado no item 3?

In [ ]:
time_inverse = timeit(lambda: newton_method_inverse(f, np.array([0, 0, 0])))[0]
time_newton = timeit(lambda: newton_method(f, np.array([0, 0, 0])))[0]

print("Time for Newton method with inverse: ", time_inverse)
print("Time for Newton method: ", time_newton)
print("Speedup: ", np.round(time_inverse / time_newton, 3))

### d) Finalmente, com os resultados a, b e R, utilizando a equação do círculo dada por $(x−a)^2 + (y −b)^2 = R^2$,
### imprima o gráfico que representa esse círculo

In [ ]:
from matplotlib import pyplot as plt, patches
fig = plt.figure()
ax = fig.add_subplot()
circle1 = patches.Circle((y[0], y[1]), radius=y[2])
ax.add_patch(circle1)
ax.axis('equal')
plt.show()
print("a: ", y[0], "b: ", y[1], "R: ", y[2])